## env

In [1]:
!nvidia-smi -L
!nvcc -V

GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU (UUID: GPU-1b776ca5-1c2d-da81-2a68-3009edfab90c)
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_19:00:59_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0


In [ ]:
import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import sys
sys.path.append('E:\Proj\Final Workspace\CNN\Code\code')
from load_data import load_MEG_dataset
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
from torch.cuda import amp
import torch.nn.functional as F
import math
# %env CUBLAS_WORKSPACE_CONFIG=:16:8
from scipy.integrate import simps
from mne.time_frequency import psd_array_welch
from band_power import bandpower_multi

In [ ]:
device = torch.device('cuda' if  torch.cuda.is_available() else 'cpu')
print(device)

# Mnet

## Load

In [ ]:
Split = 0.10
X_train, y_train = load_MEG_dataset([str(i).zfill(3) for i in range(1,2)], mode = 'concatenate', output_format='numpy',shuffle = False, training=True, train_test_split=Split, batch_size=500)
X_test, y_test = load_MEG_dataset([str(i).zfill(3) for i in range(1,2)], mode = 'concatenate', output_format='numpy',shuffle = False, training=False, train_test_split=Split, batch_size=500)

In [ ]:
X_train = X_train[:, None, ...]
X_test = X_test[:, None, ...]
y_train = (y_train / 2).astype(int) - 1
y_test = (y_test / 2).astype(int) - 1

In [ ]:
# X_train=np.repeat(X_train,8,axis=3)
# X_test=np.repeat(X_test,8,axis=3)

In [ ]:
X_train_tensors = torch.Tensor(X_train)
X_test_tensors = torch.Tensor(X_test)
y_train_tensors = torch.from_numpy(y_train.astype(int)) # for cross entropy loss
y_test_tensors = torch.from_numpy(y_test.astype(int))

In [ ]:
X_train.shape

In [ ]:
#reshaping to rows, timestamps, features
# X_train_tensors = torch.permute(X_train_tensors, (3,1,2,0))
# X_test_tensors = torch.permute(X_test_tensors, (3,1,2,0)) 

In [ ]:
def onehot(batches, n_classes, y):
  yn = torch.zeros(batches, n_classes)
  for i in range(batches):
    x = [0 for j in range(batches)]
    x[i] = y[i]/2-1                     #ex. [12]-> [5]
    yn[i][int(x[i])]+= 1                  #[000010000]
  return yn

In [ ]:
# y_train_tensors_onehot = onehot(int(y_train_tensors.size(0)), len(np.unique(y_train)), y_train_tensors)
# y_test_tensors_onehot = onehot(int(y_test_tensors.size(0)), len(np.unique(y_train)), y_test_tensors)
# print(y_train_tensors_onehot.shape)

In [ ]:
X_train_tensors=X_train_tensors.cuda()
X_test_tensors=X_test_tensors.cuda()
y_train_tensors=y_train_tensors.cuda()
y_test_tensors=y_test_tensors.cuda()

Test for band power

In [ ]:
X = np.swapaxes(X_train, 2, -1).squeeze()
data = X[80, 20, :]
print(data.shape)

## band power 

In [ ]:
psd_mne, freqs_mne = psd_array_welch(data, 250, 1., 70., n_per_seg=None,
                          n_overlap=0, n_jobs=1)
for low, high in [(1, 4), (4, 8), (8, 10), (10, 13), (13, 30),
                  (30, 70)]:
    print("processing bands (low, high) : ({},{})".format(low, high))
    # Find intersecting values in frequency vector
    idx_delta = np.logical_and(freqs_mne >= low, freqs_mne <= high)
      # Frequency resolution
    freq_res = freqs_mne[1] - freqs_mne[0]  # = 1 / 4 = 0.25

    # Compute the absolute power by approximating the area under the curve
    power = simps(psd_mne[idx_delta], dx=freq_res)
    print('Absolute power: {:.4f} uV^2'.format(power))
    
    total_power = simps(psd_mne, dx=freq_res)
    rel_power = power / total_power
    
    print('Relative power: {:.4f}'.format(rel_power))

In [ ]:
def bandpower_1d(data, sf, band, nperseg=None, relative=False):
    """
        Compute the average power of the signal x in a specific frequency band.
        https://raphaelvallat.com/bandpower.html
    Args:
        data (1d-array):
            Input signal in the time-domain.
        sf (float):
            Sampling frequency of the data.
        band (list):
            Lower and upper frequencies of the band of interest.
        window_sec (float):
            Length of each window in seconds.
            If None, window_sec = (1 / min(band)) * 2
        relative (boolean):
            If True, return the relative power (= divided by the total power of the signal).
            If False (default), return the absolute power.

    Returns:
        bp (float):
            Absolute or relative band power.
    """

    # band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    psd, freqs = psd_array_welch(data, sf, 1., 70., n_per_seg=None,
                          n_overlap=0, n_jobs=1)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp

In [ ]:
def bandpower(x, fs, fmin, fmax, nperseg=None, relative=True):
    """
    Compute the average power of the multi-channel signal x in a specific frequency band.
    Args:
        x (nd-array): [n_epoch, n_channel, n_times]
           The epoched input data.
        fs (float):
            Sampling frequency of the data.
        fmin (int): Low-band frequency.
        fmax (int): High-band frequency.
        window_sec (float):
            Length of each window in seconds.
            If None, window_sec = (1 / min(band)) * 2
        relative (boolean):
            If True, return the relative power (= divided by the total power of the signal).
            If False (default), return the absolute power.

    Returns:
        bp (nd-array): [n_epoch, n_channel, 1]
            Absolute or relative band power.
    """
    n_epoch, n_channel, _ = x.shape

    bp = np.zeros((n_epoch, n_channel, 1))
    for epoch in range(n_epoch):
        for channel in range(n_channel):
            bp[epoch, channel] = bandpower_1d(
                x[epoch, channel, :],
                fs,
                [fmin, fmax],
                nperseg=nperseg,
                relative=relative,
            )

    return bp

In [ ]:
def bandpower_multi(x, fs, bands,  nperseg=None, relative=True):
    """
    Compute the average power of the multi-channel signal x in multiple frequency bands.
    Args:
        x (nd-array): [n_epoch, n_channel, n_times]
           The epoched input data.
        fs (float):
            Sampling frequency of the data.
        bands (list): list of bands to compute the bandpower. echa band is a tuple of fmin and fmax.
        window_sec (float):
            Length of each window in seconds.
            If None, window_sec = (1 / min(band)) * 2
        relative (boolean):
            If True, return the relative power (= divided by the total power of the signal).
            If False (default), return the absolute power.

    Returns:
        bp (nd-array): [n_epoch, n_channel, n_bands]
            Absolute or relative bands power.
    """
    n_epoch, n_channel, _ = x.shape
    bp_list = []
    for idx, band in enumerate(bands):
        fmin, fmax = band
        bp_list.append(
            bandpower(
                x, fs, fmin, fmax,  nperseg=None, relative=relative
            )
        )

    bp = np.concatenate(bp_list, -1)

    return bp

In [ ]:
X_train.shape

In [ ]:
X_train_tensors.shape

In [ ]:
X_train_bp = np.squeeze(X_train, axis=1)
X_train_bp = X_train_bp[: 10, :, :]

In [ ]:
import numpy as np


In [ ]:
bands = [(1, 4), (4, 8), (8, 10), (10, 13), (13, 30), (30, 70)]
bp = bandpower_multi(
    X_train_bp, fs=100.0, bands=bands, relative=True
    )

## Parms

In [ ]:
CRED    = '\33[31m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CBLUE   = '\33[34m'

## model

In [ ]:
class ChannelPool(nn.Module):

    def forward(self, x):
        return torch.cat((torch.max(x, 1)[0].unsqueeze(1),
                          torch.mean(x, 1).unsqueeze(1)), dim=1)

In [ ]:
class SpatialAttention(nn.Module):
  def __init__(self):
    super(SpatialAttention, self).__init__()
    self.spatialAttention = nn.Sequential(
        ChannelPool(),
        nn.Conv2d(2, 1, 7, 7, padding=3),
        nn.Sigmoid(),
        )

  def forward(self, x):
    return x * self.spatialAttention(x)


In [ ]:
class Flatten_MEG(nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

In [ ]:
class ChannelAttention(nn.Module):
    """
            Implementation of a channel attention module.
        """

    def __init__(self, shape, reduction_factor=16):

        super(ChannelAttention, self).__init__()

        _, in_channel, h, w = shape

        self.mlp = nn.Sequential(
            Flatten_MEG(),
            nn.Linear(in_channel, in_channel // reduction_factor),
            nn.ReLU(),
            nn.Linear(in_channel // reduction_factor, in_channel),
        )
        self.avg = nn.AvgPool2d(kernel_size=(h, w), stride=(h, w))
        self.max = nn.MaxPool2d(kernel_size=(h, w), stride=(h, w))

    def forward(self, x):

        avg = self.avg(x)
        max = self.max(x)

        attention = (
            torch.sigmoid(self.mlp(avg) + self.mlp(max))
            .unsqueeze(2)
            .unsqueeze(3)
        )

        return x * attention


In [ ]:
class MNet(nn.Module):
    """
        Model inspired by [Aoe at al., 10.1038/s41598-019-41500-x]
    """

    def __init__(self, n_times):
        """
        Args:
            n_times (int):
                n_times dimension of the input data.
        """
        super(MNet, self).__init__()
        self.n_times = n_times
        # if n_times == 501:  # TODO automatic n_times
        #     self.n_times = 12
        # elif n_times == 601:
        #     self.n_times = 2
        # elif n_times == 701:
        #     self.n_times = 4
        # else:
        #     raise ValueError("Network can work only with n_times = 501, 601, "
        #                      "701 (epoch duration of 1., 1.2, 1.4 sec),"
        #                      " got instead {}".format(n_times))

        self.spatial = nn.Sequential(
            nn.Conv2d(1, 32, stride=(1,2), kernel_size=(272,64), bias=False), #kernel size 204, 64
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=(1, 16), bias=False), # kernel size 1,16
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.BatchNorm2d(64),
        )

        self.temporal = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(8, 8), bias=False),
            nn.ReLU(),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size=(8, 8), bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 3), stride=(1, 2)),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=(6, 6), bias=False),
            nn.ReLU(),
            # nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=(6, 6), bias=False),
            nn.ReLU(),
            # nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.Conv2d(64, 128, kernel_size=(5, 5), bias=False),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=(5, 5), bias=False),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.Conv2d(128, 256, kernel_size=(4, 4), bias=False),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=(4, 4), bias=False),
            nn.ReLU(),
            nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(256),
        )

        self.attention = nn.Sequential(
            ChannelAttention([None, 256, 26, self.n_times]), SpatialAttention()
        )

        self.flatten = Flatten_MEG()

        self.ff = nn.Sequential(
            nn.Linear(256 * 26 * self.n_times, 1024),
            nn.BatchNorm1d(num_features=1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(num_features=1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 14),
        )

    def forward(self, x):
        x = self.spatial(x)
        print(x.shape)
        x = torch.transpose(x, 1, 2)
        print(x.shape)        
        x = self.temporal(x)
        # x = self.attention(x)
        x = self.ff(self.flatten(x))

        return x.squeeze(1)

In [ ]:
mnet = MNet(11).cuda()
print(mnet)

## original Aoe model

In [ ]:
class Concatenate(nn.Module):
    def __init__(self):
        super(Concatenate, self).__init__()

    def forward(self, x, bp):

        # min_ = x.min(1, keepdim=True)[0]
        # if min_[0] < 0:
        #     x = x + min_
        # else:
        #     x = x - min_
        # x = x / x.max()
        x = x.view(x.shape[0], -1)
        bp = bp.view(bp.shape[0], -1)
        x = torch.cat([x, bp], -1)

        return x

In [ ]:
class AoeMNet(nn.Module):
    """
        Model inspired by [Aoe at al., 10.1038/s41598-019-41500-x]
    """
    def __init__(self, n_times):
        super(AoeMNet, self).__init__()
        self.n_times = n_times
        self.spatial = nn.Sequential(
            nn.Conv2d(1, 32, stride=(1,2), kernel_size=(272,64), bias=False), #kernel size 204, 64
            nn.ReLU(),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, stride=(1,2), kernel_size=(1, 16), bias=False), # kernel size 1,16
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            # nn.BatchNorm2d(64),
        )

        self.temporal = nn.Sequential(
            nn.Conv2d(1, 32, stride=(1, 1), kernel_size=(8, 8), bias=False),
            nn.ReLU(),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, stride=(1, 1), kernel_size=(8, 8), bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(5, 3), stride=(5, 3)),
            # nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, stride=(1, 1), kernel_size=(1, 4), bias=False),
            nn.ReLU(),
            # nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, stride=(1, 1), kernel_size=(1, 4), bias=False), #conv6
            nn.ReLU(),
            # nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.Conv2d(64, 128, stride=(1, 1), kernel_size=(1, 2), bias=False),
            nn.ReLU(),
            # nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, stride=(1, 1), kernel_size=(1, 2), bias=False),
            nn.ReLU(),
            # nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.Conv2d(128, 256, stride=(1, 1), kernel_size=(1, 2), bias=False),
            nn.ReLU(),
            # nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, stride=(1, 1), kernel_size=(1, 2), bias=False), #conv10
            nn.ReLU(),
            # nn.Dropout2d(p=0.3),
            # nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),

        )

        self.attention = nn.Sequential(
            ChannelAttention([None, 256, 26, self.n_times]), SpatialAttention()
        )

        self.flatten = Flatten_MEG()
        # self.concatenate = Concatenate()

        self.ff = nn.Sequential(
            nn.Linear(64 * 2 * self.n_times + 272 * 6, 1024),
            # nn.Linear(5120,1024),
            # nn.BatchNorm1d(num_features=1024),
            nn.ReLU(),
            # nn.Dropout(0.3),
            nn.Linear(1024, 1024),
            # nn.BatchNorm1d(num_features=1024),
            nn.ReLU(),
            # nn.Dropout(0.3),
            nn.Linear(1024, 14),
        )
    def forward(self, x):
        x = self.spatial(x)
        # print(x.shape)
        x = torch.transpose(x, 1, 2)
        # print(x.shape)        
        x = self.temporal(x)
        x = self.attention(x)
        # print(x.shape)
        x = self.flatten(x)
        # print(x.shape)
        x = self.ff(x)

        return x.squeeze(1)
    # def forward(self, x, pb):
    #     x = self.spatial(x)
    #     x = torch.transpose(x, 1, 2)
    #     x = self.temporal(x)
    #     x = self.concatenate(x)
    #     x = self.ff(self.flatten(x))

    #     return x.squeeze(1)

In [ ]:
aoemnet = AoeMNet(11).cuda()
print(aoemnet)

In [ ]:
def GETcorrectnumber(loader, printcolor):
  with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]
    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = aoemnet(inputs)
        optimizer.step()          
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        # n_correct += (predicted == labels).sum().item()
        for k in range(predicted.shape[0]):
          if predicted[k]==labels[k]:
            n_correct +=1
        # for i in range(num_classes): # accuracy for each class
        #     label = labels[i]
        #     pred = predicted[i]
        #     if (label == pred):
        #         n_class_correct[i] += 1
        #     n_class_samples[i] += 1
    acc = 100.0 * n_correct / n_samples
    print(printcolor+f'[{epoch + 1}] t accuracy： {acc}%'+printcolor)
  return acc

## Train

In [ ]:
BATCH_SIZE=32
train = Data.TensorDataset(X_train_tensors, y_train_tensors)
test = Data.TensorDataset(X_test_tensors, y_test_tensors)
train_loader = Data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = Data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

learning_rate = 0.0001
criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(mnet.parameters(), lr=learning_rate) 
optimizer = torch.optim.SGD(mnet.parameters(), lr=learning_rate) 

In [ ]:
from torch.utils.tensorboard import SummaryWriter
summary_writer = SummaryWriter(f'./models/test')
train_loss = []
valid_loss = []
aoemnet.train()
for epoch in range(num_epochs):  # loop over the dataset multiple times
  t_loss = 0
  for i, data in enumerate(train_loader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    # print(inputs.shape)
    inputs, labels = inputs.to(device), labels.to(device)
    # zero the parameter gradients
    optimizer.zero_grad()
    # forward + backward + optimize
    outputs = aoemnet(inputs)
    # with torch.autocast('cuda'):
    # loss = criterion(outputs, torch.tensor(labels).cuda())
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    t_loss += loss.item()
    # print(t_loss)
    train_loss.append(np.mean(t_loss))

    if epoch % 1 == 0:
      if i % (math.ceil(900*Split/BATCH_SIZE)-1) == 0 and i !=0:
          print(CRED+ f'[{epoch + 1}, {i + 1}] trainning loss: {train_loss[-1]}'+ CRED)
  summary_writer.add_scalar('train_loss', train_loss[-1], epoch)

  if epoch % 1 == 0:
    aoemnet.eval()
    va_loss = 0
    for i, data in enumerate(test_loader, 0):
      val_x, val_y = data
      val_x, val_y = val_x.to(device), val_y.to(device)
      Testoutput = aoemnet(val_x)
      # v_loss = criterion(Testoutput, val_y, torch.Tensor(Testoutput.size(0)).cuda().fill_(1.0))
      v_loss = criterion(Testoutput, val_y) #loss
      va_loss += v_loss.item()
      valid_loss.append(np.mean(va_loss))
      if i % (math.ceil(900*(1-Split)/BATCH_SIZE)-1) == 0 and i !=0:    # print every first
          print(CGREEN+f'[{epoch + 1}, {i + 1}] valid_loss： {valid_loss[-1]}'+CGREEN)
  summary_writer.add_scalar('valid_loss', valid_loss[-1], epoch)

  # if epoch % 1 == 0:
  #   GETcorrectnumber(train_loader,CYELLOW)      
  #   GETcorrectnumber(test_loader,CBLUE)

  # aoemnet.train()
print('Finished Training')

Confusion matrix and mean accuracy

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
confusion_matrix_test_loader = Data.DataLoader(test, batch_size = 90, shuffle = True)
with torch.no_grad():
    list_mean_accuracy = []
    for inputs, labels in confusion_matrix_test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = aoemnet(inputs)
        optimizer.step()          
        _, predicted = torch.max(outputs, 1)
        labels = labels.cpu()
        predicted = predicted.cpu()
        labels = labels.numpy()
        predicted = predicted.numpy()
        mean_conf_mat = confusion_matrix(labels, predicted)
        mean_accuracy = accuracy_score(labels[labels != 99], predicted[predicted != 99])
        mean_conf_mat = mean_conf_mat.astype('float') / mean_conf_mat.sum(axis=1)  # normalise
        list_mean_accuracy.append(mean_accuracy)
        print("Mean accuracy = {0}".format(mean_accuracy))
        ConfusionMatrixDisplay.from_predictions(labels, predicted)
        # plt.savefig('/content/drive/MyDrive/MT_ML_Decoding/Aversive_state_reactivation/notebooks/templates/save_folder/fig-{}.png'.format(session_id), dpi=600)
        plt.show()

In [ ]:
# !rm -rf /logs/ # clear logs
if 'google.colab' in str(get_ipython()): # tensor board
  %load_ext tensorboard  
  # %tensorboard --logdir logs
  %tensorboard --logdir=./models/test